In [1]:
import pandas as pd

In [2]:

data = pd.read_csv('data/processed_data.csv',index_col='id')
reviews = list(data['tweet'].apply(str))
labels = list(data['label'])

In [3]:
for sent in reviews:
    sent.split()

In [4]:
corpus = [word for sent in reviews for word in str(sent).split()]

In [5]:
word2index = {}
for i,word in enumerate(corpus):
    word2index[word] = i
word2index

{'when': 245233,
 'father': 244190,
 'dysfunctional': 2,
 'selfish': 214172,
 'drags': 238952,
 'kids': 239848,
 'into': 244014,
 'dysfunction': 194202,
 '#run': 235279,
 'thanks': 245060,
 '#lyft': 194206,
 'credit': 229345,
 'cause': 235175,
 'they': 245090,
 'offer': 243743,
 'wheelchair': 64453,
 'vans': 16,
 '#disapointed': 17,
 '#getthanked': 79030,
 'bihday': 243917,
 'your': 245227,
 'majesty': 205876,
 '#model': 245144,
 'love': 245149,
 'take': 244843,
 'with': 245210,
 'time': 245243,
 'factsguide': 240977,
 'society': 243471,
 '#motivation': 243184,
 'huge': 237781,
 'fare': 161248,
 'talking': 234357,
 'before': 244130,
 'leave': 244746,
 'chaos': 221455,
 'disputes': 37,
 'there': 245085,
 '#allshowandnogo': 41,
 'camping': 234512,
 'tomorrow': 245239,
 'danny': 242048,
 'next': 244850,
 'school': 244230,
 'year': 245125,
 'exams': 233800,
 'think': 244204,
 'about': 244328,
 'that': 245245,
 '#school': 236807,
 '#exams': 220707,
 '#hate': 239356,
 '#imagine': 235373,
 '#

In [19]:
import time
import sys
import numpy as np


class SarcasmNetwork:
    def __init__(self, reviews,labels,hidden_nodes = 10, learning_rate = 0.1):
       
        self.pre_process_data(reviews, labels)
      
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        
        
        review_vocab = set()
        for review in reviews:
            for word in review.split(" "):
                review_vocab.add(word)

        self.review_vocab = list(review_vocab)
        

        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
  
        self.label_vocab = list(label_vocab)
        
     
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
     
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
    
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
  
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.learning_rate = learning_rate
        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))
    
        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        self.layer_0 = np.zeros((1,input_nodes))
    
    def update_input_layer(self,review):

        self.layer_0 *= 0
        
        for word in review.split(" "):

            if(word in self.word2index.keys()):
                self.layer_0[0][self.word2index[word]] += 1
                
    def get_target_for_label(self,label):
        if(label == 1):
            return 1
        else:
            return 0
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
    def train(self, training_reviews, training_labels):

        assert(len(training_reviews) == len(training_labels))


        start = time.time()

        for i in range(len(training_reviews)):
    
            review = training_reviews[i]
            label = training_labels[i]
   
            self.update_input_layer(review)

     
            layer_1 = self.layer_0.dot(self.weights_0_1)

        
            layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
            
     
            layer_2_error = layer_2 - self.get_target_for_label(label) 
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)


            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) 
            layer_1_delta = layer_1_error 
            self.weights_1_2 -= layer_1.T.dot(layer_2_delta) * self.learning_rate 
            self.weights_0_1 -= self.layer_0.T.dot(layer_1_delta) * self.learning_rate 


            
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5])
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):

        correct = 0

      
        start = time.time()

        
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
        

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
       
       
        self.update_input_layer(review.lower())

        layer_1 = self.layer_0.dot(self.weights_0_1)

        layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
        
        
        if(layer_2[0] >= 0.5):
            return 1
        else:
            return 0
        

In [20]:
model_1 = SarcasmNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)

In [21]:
model_1.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0
Progress:8.07% Speed(reviews/sec):120.5
Progress:16.1% Speed(reviews/sec):120.9
Progress:24.2% Speed(reviews/sec):121.3
Progress:32.2% Speed(reviews/sec):121.2
Progress:40.3% Speed(reviews/sec):121.3
Progress:48.4% Speed(reviews/sec):120.4
Progress:56.5% Speed(reviews/sec):120.2
Progress:64.5% Speed(reviews/sec):120.2
Progress:72.6% Speed(reviews/sec):120.1
Progress:80.7% Speed(reviews/sec):120.0
Progress:88.8% Speed(reviews/sec):120.1
Progress:96.8% Speed(reviews/sec):120.0
Progress:99.9% Speed(reviews/sec):120.0

In [22]:
model_1.test(reviews[-1000:],labels[-1000:])

Progress:16.5% Speed(reviews/sec):999.9 #Correct:162 #Tested:166 Testing Accuracy:97.5%

Progress:99.9% Speed(reviews/sec):1015. #Correct:951 #Tested:1000 Testing Accuracy:95.1%

In [26]:
import time
import sys
import numpy as np


class SarcasmNetworkNew:
    def __init__(self, reviews,labels,hidden_nodes = 10, learning_rate = 0.1):
     
        self.pre_process_data(reviews, labels)
       
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
       
        review_vocab = set()
        for review in reviews:
            for word in review.split(" "):
                review_vocab.add(word)

   
        self.review_vocab = list(review_vocab)
     
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
    
        self.label_vocab = list(label_vocab)
        
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
       
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i

    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
     
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

      
        self.learning_rate = learning_rate

        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))

    
        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        self.layer_1 = np.zeros((1,hidden_nodes))
    
    def get_target_for_label(self,label):
        if(label == 1):
            return 1
        else:
            return 0
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
  
    def train(self, training_reviews_raw, training_labels):

    
        training_reviews = list()
        for review in training_reviews_raw:
            indices = set()
            for word in review.split(" "):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))

      
        assert(len(training_reviews) == len(training_labels))
   
        start = time.time()
   
        for i in range(len(training_reviews)):
            
            review = training_reviews[i]
            label = training_labels[i]
            
            
            self.layer_1 *= 0
            for index in review:
                self.layer_1 += self.weights_0_1[index]

          
            layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))            
            
          
            layer_2_error = layer_2 - self.get_target_for_label(label) # Output layer error is the difference between desired target and actual output.
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)

       
            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) 
            layer_1_delta = layer_1_error

        
            self.weights_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate 
            
         
            for index in review:
                self.weights_0_1[index] -= layer_1_delta[0] * self.learning_rate 
                        
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5])
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
       
       
        correct = 0

        start = time.time()

        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
           
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        self.layer_1 *= 0
        unique_indices = set()
        for word in review.lower().split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += self.weights_0_1[index]
        
        
        layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
        
        if(layer_2[0] >= 0.5):
            return 1
        else:
            return 0

In [24]:
model_2 = SarcasmNetworkNew(reviews[:-1000],labels[:-1000], learning_rate=0.1)
model_2.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0


Progress:6.56% Speed(reviews/sec):4747.

Progress:8.07% Speed(reviews/sec):4132.


Progress:10.0% Speed(reviews/sec):3894.

Progress:12.8% Speed(reviews/sec):3963.

...

Progress:88.8% Speed(reviews/sec):4188.


Progress:92.5% Speed(reviews/sec):4201.

Progress:94.6% Speed(reviews/sec):4185.

Progress:96.8% Speed(reviews/sec):4189.


Progress:99.9% Speed(reviews/sec):4199.

In [25]:
model_2.test(reviews[-1000:],labels[-1000:])

Progress:89.2% Speed(reviews/sec):5068. #Correct:850 #Tested:893 Testing Accuracy:95.1%

Progress:99.9% Speed(reviews/sec):5285. #Correct:948 #Tested:1000 Testing Accuracy:94.8%